#

# LP reader

This Notebook develops a .lp reader.

In [ ]:
function read_file_to_string(file_path::String)
    return open(file_path, "r") do f
        read(f, String)
    end
end

In [1]:
using SparseArrays
using DataStructures: OrderedDict


struct MIPProblem
    is_minimize::Bool
    c::Vector{Float64}
    A::SparseMatrixCSC{Float64, Int64}
    b::Vector{Float64}
    l::Vector{Float64}
    u::Vector{Float64}
    vars::Vector{String}
    variable_types::Vector{Symbol}
    constraint_types::Vector{Char}
end

In [33]:
function debug_print(debug::Bool, msg...)
    if debug
        println(msg...)
    end
end

debug_print (generic function with 1 method)

In [31]:
function read_lp_from_string(lp_string::String; debug::Bool=false)
    # Initialize fields
    debug_print(debug, "Starting read_lp_from_string...")
    
    is_minimize = true
    c = OrderedDict{String, Float64}()  # Keep track of objective coefficients
    A_data = Float64[]
    A_row = Int[]
    A_col = Int[]
    b = Float64[]
    l = OrderedDict{String, Float64}()
    u = OrderedDict{String, Float64}()
    vars = OrderedDict{String, Nothing}()  # Ensure consistent variable order
    variable_types = OrderedDict{String, Symbol}()
    constraint_types = Char[]

    # Split the LP string into lines and start parsing
    lines = split(lp_string, "\n")
    debug_print(debug, "Parsed lines: ", lines)

    # Parse objective sense (Minimize or Maximize)
    for line in lines
        if startswith(line, "Minimize")
            is_minimize = true
            debug_print(debug, "Objective is minimization.")
        elseif startswith(line, "Maximize")
            is_minimize = false
            debug_print(debug, "Objective is maximization.")
        end
    end

    in_objective_section = false
    in_constraint_section = false
    in_bounds_section = false

    for line in lines
        line = strip(line)
        if isempty(line)
            continue
        end

        # Debug: Current line being processed
        debug_print(debug, "Processing line: ", line)

        # Parse the objective function
        if startswith(line, "Minimize") || startswith(line, "Maximize")
            in_objective_section = true
            debug_print(debug, "Entering objective function section.")
            continue
        end

        if in_objective_section
            if startswith(line, "Subject To")
                in_objective_section = false
                in_constraint_section = true
                debug_print(debug, "Finished objective function, entering constraints section.")
                continue
            end

            # Remove the prefix 'obj:' from the line
            if startswith(line, "obj:")
                line = strip(line[5:end])  # Remove "obj: " prefix
            end

            # Ensure both positive and negative coefficients are handled
            terms = split(replace(strip(line), "-" => "+-"), " + ")
            debug_print(debug, "Objective terms: ", terms)

            for term in terms
                parts = split(term, " ")
                if length(parts) == 2
                    coefficient = tryparse(Float64, parts[1])
                    variable = parts[2]
                    if coefficient !== nothing && !isempty(variable)
                        debug_print(debug, "Adding to objective: ", variable, " -> ", coefficient)
                        c[variable] = coefficient  # Assign coefficients to the objective
                        if !(haskey(vars, variable))
                            vars[variable] = nothing  # Track variable order in vars
                            variable_types[variable] = :Continuous  # Default variable type
                            l[variable] = -Inf  # Default lower bound
                            u[variable] = Inf   # Default upper bound
                            debug_print(debug, "Added variable to vars: ", variable)
                        end
                    end
                end
            end
        end

        # Parse constraints
        if in_constraint_section
            if startswith(line, "Bounds")
                in_constraint_section = false
                in_bounds_section = true
                debug_print(debug, "Entering bounds section.")
                continue
            end

            if contains(line, "<=") || contains(line, ">=") || contains(line, "=")
                terms = split(replace(strip(line), "+" => ""), " ")
                debug_print(debug, "Constraint terms: ", terms)
                row = length(b) + 1

                # Parse each variable and coefficient in the constraint
                i = 2  # Start after 'c1:' or 'c2:'
                while i < length(terms) - 2
                    coefficient = tryparse(Float64, terms[i])
                    variable = terms[i+1]
                    if coefficient !== nothing && !isempty(variable)
                        col = findfirst(collect(keys(vars)) .== variable)
                        if col === nothing
                            vars[variable] = nothing
                            variable_types[variable] = :Continuous
                            l[variable] = -Inf
                            u[variable] = Inf
                            col = length(vars)
                        end
                        push!(A_data, coefficient)
                        push!(A_row, row)
                        push!(A_col, col)
                        debug_print(debug, "Added constraint coefficient: ", coefficient, " for variable: ", variable)
                    end
                    i += 2  # Move to next variable-coefficient pair
                end

                # Parse the RHS of the constraint
                rhs = tryparse(Float64, terms[end])
                if rhs !== nothing
                    push!(b, rhs)
                    debug_print(debug, "Added RHS: ", rhs)
                    if contains(line, "<=")
                        push!(constraint_types, 'L')
                    elseif contains(line, ">=")
                        push!(constraint_types, 'G')
                    else
                        push!(constraint_types, 'E')
                    end
                end
            end
        end

        # Parse bounds
        if in_bounds_section
            if startswith(line, "End")
                debug_print(debug, "Reached end of file.")
                break
            end
            terms = split(line, " ")
            debug_print(debug, "Bounds terms: ", terms)

            if length(terms) == 5
                lower_bound = tryparse(Float64, terms[1])
                upper_bound = tryparse(Float64, terms[5])
                variable = terms[3]
                if !(haskey(vars, variable))
                    vars[variable] = nothing
                    variable_types[variable] = :Continuous
                    l[variable] = -Inf
                    u[variable] = Inf
                end
                if lower_bound !== nothing && upper_bound !== nothing
                    l[variable] = lower_bound
                    u[variable] = upper_bound
                    debug_print(debug, "Added bounds: ", variable, " -> [", lower_bound, ", ", upper_bound, "]")
                end
            elseif length(terms) == 3
                bound = tryparse(Float64, terms[1])
                variable = terms[3]
                if !(haskey(vars, variable))
                    vars[variable] = nothing
                    variable_types[variable] = :Continuous
                    l[variable] = -Inf
                    u[variable] = Inf
                end
                if bound !== nothing
                    if startswith(line, ">= ")
                        l[variable] = bound
                    elseif startswith(line, "<= ")
                        u[variable] = bound
                    end
                    debug_print(debug, "Added bound: ", variable, " -> ", bound)
                end
            end
        end
    end

    # Create consistent vectors for l, u, and c based on the order of vars
    debug_print(debug, "Final vars order: ", keys(vars))
    c_vec = [get(c, var, 0.0) for var in keys(vars)]  # Ensure correct order for coefficients
    l_vec = [l[var] for var in keys(vars)]
    u_vec = [u[var] for var in keys(vars)]

    debug_print(debug, "Final objective vector (c): ", c_vec)
    debug_print(debug, "Final lower bounds (l): ", l_vec)
    debug_print(debug, "Final upper bounds (u): ", u_vec)

    # Create sparse matrix for A
    A = sparse(A_row, A_col, A_data, length(b), length(c_vec))
    debug_print(debug, "Sparse matrix A: ", A)

    # Convert variable_types Dict to a Vector
    variable_types_vec = [variable_types[var] for var in keys(vars)]

    # Return the constructed MIPProblem
    debug_print(debug, "Returning MIPProblem.")
    return MIPProblem(is_minimize, c_vec, A, b, l_vec, u_vec, collect(keys(vars)), variable_types_vec, constraint_types)
end


read_lp_from_string (generic function with 1 method)

In [36]:
lp_string = """
Maximize
 obj: 1 X1 + 1 X2
Subject To
 c1: 2 X1 + 3 X2 <= 10
 c2: 2 X1 + X2 >= 5
Bounds
 0 <= X1 <= 10
 0 <= X2 <= 1
End
"""

problem = read_lp_from_string(lp_string,debug=true)


Starting read_lp_from_string...
Parsed lines: SubString{String}["Maximize", " obj: 1 X1 + 1 X2", "Subject To", " c1: 2 X1 + 3 X2 <= 10", " c2: 2 X1 + X2 >= 5", "Bounds", " 0 <= X1 <= 10", " 0 <= X2 <= 1", "End", ""]
Objective is maximization.
Processing line: Maximize
Entering objective function section.
Processing line: obj: 1 X1 + 1 X2
Objective terms: SubString{String}["1 X1", "1 X2"]
Adding to objective: X1 -> 1.0
Added variable to vars: X1
Adding to objective: X2 -> 1.0
Added variable to vars: X2
Processing line: Subject To
Finished objective function, entering constraints section.
Processing line: c1: 2 X1 + 3 X2 <= 10
Constraint terms: SubString{String}["c1:", "2", "X1", "+", "3", "X2", "<=", "10"]
Added constraint coefficient: 2.0 for variable: X1
Added RHS: 10.0
Processing line: c2: 2 X1 + X2 >= 5
Constraint terms: SubString{String}["c2:", "2", "X1", "+", "X2", ">=", "5"]
Added constraint coefficient: 2.0 for variable: X1
Added RHS: 5.0
Processing line: Bounds
Entering bounds 

MIPProblem(false, [1.0, 1.0, 0.0], sparse([1, 2], [1, 1], [2.0, 2.0], 2, 3), [10.0, 5.0], [-Inf, -Inf, -Inf], [Inf, Inf, Inf], ["X1", "X2", "<="], [:Continuous, :Continuous, :Continuous], ['L', 'G'])

In [23]:
problem

MIPProblem(false, [1.0, 1.0], sparse([1, 2], [1, 1], [2.0, 2.0], 2, 2), [10.0, 5.0], [0.0, 0.0], [10.0, 1.0], ["X1", "X2"], [:Continuous, :Continuous], ['L', 'G'])

## Checking against Highs ones

In [25]:
mps_folder_path = "../check/problems/ls_files/"

"../check/problems/ls_files/"